[View in Colaboratory](https://colab.research.google.com/github/alvarogutyerrez/alvarogutyerrez/blob/master/S07_Modelos_Lineales_Regularizados.ipynb)

#Sesión 7: Modelos Lineales Regularizados 

En esta sesión, aprenderemos como, haciendo uso de sklearn, estimar modelos de regresión lineal regulariazados (e.g Lasso, Ridge, ElasticNet). Adicional a esto, declararemos una serie de funciones que serán de utilidad en la generación de rezagos en la data (**crealags**), para generar listas ordenadas con todos los modelos que queramos estimar (**creamodelos**),  y finalmente otra que será útil para la presentación de las métricas de desempeño de dichos modelos en cada una de las muestras (**metricas**).

Primero cargaremos la base de datoa a utilizar.

In [57]:
!wget https://www.dropbox.com/s/3vo2v28ho7o59ms/data_usd-clp.xlsx
!pip install xlrd


Redirecting output to ‘wget-log.3’.


In [0]:
import pandas as pd
excel_file= 'data_usd-clp.xlsx'
data = pd.read_excel(excel_file)

##Estimacion Modelos Regularizados

Desde la librería sklearn importaremos todos los modelos que utilizaremos en esta sesión, así como las métricas necesarias.

In [0]:
''' MODELOS a Estimar '''
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
'''METRICS'''
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error



##Separando Muestras (Training v/s Testing)


Como es usual, definiendo una fecha de corte. Separaremos la data en dos muestras. Una de entrenamiento (training) y otra para evaluar el desempeño de dichos modelos en una data diferente a la que fueron entrenados (testing)

In [0]:
#Separando Explicada de Explicativas
y= data.iloc[:,0]
x= data.iloc[:,1:-1]
#Umbral de Corte
year_corte = '2016-06-19 00:00:00'
#Trainig
x_tr =x[x.index<year_corte].copy(True)
y_tr =y[y.index<year_corte].copy(True)
#Testing
x_tst =x[x.index>year_corte].copy(True)
y_tst =y[y.index>year_corte].copy(True)

##Plain Vanilla Models

Partiremos estimando (en la muestra de entrenamiento) los modelos regularizados (Lasso, Ridge, ElasticNet) para ver el comportamiento de los parámetros. De los resultados vemos como algunos regresores en los modelos regularizados, fueron "empujados" hacia cero.


In [61]:
#Declarando modelos
linear_reg= LinearRegression()
reg_ridge=  Ridge()
reg_lasso= Lasso()
reg_elastic = ElasticNet()
#Ajustando Modelos
linear_reg.fit(x_tr,y_tr)
reg_ridge.fit(x_tr,y_tr)
reg_lasso.fit(x_tr,y_tr)
reg_elastic.fit(x_tr,y_tr)
#Recuperando Betas
coef_linear_reg = pd.DataFrame(linear_reg.coef_,index = x_tr.columns , columns = {'LinearRegression'})
coef_ridge = pd.DataFrame(reg_ridge.coef_ ,index = x_tr.columns , columns = {'Ridge'})
coef_lasso =pd.DataFrame(reg_lasso.coef_,index = x_tr.columns , columns = {'Lasso'})
coef_elastic_net = pd.DataFrame(reg_elastic.coef_,index = x_tr.columns , columns = {'ElasticNet'})
#Consolidando Betas
coef =   coef_ridge, coef_lasso , coef_elastic_net
compendio=pd.concat([coef_linear_reg], axis=1).copy(True)
for k in coef:
  compendio=pd.concat([compendio,k ], axis=1).copy(True)
  
compendio

,LinearRegression,Ridge,Lasso,ElasticNet
USSW1 Curncy,90.340647,65.411975,0.000000,-0.000000
USSW10 Curncy,-2.754454,0.520155,0.000000,3.087177
CHSWP1 ICCH Curncy,-48.189200,-42.699925,-25.853362,-8.989084
CHSWP10 Curncy,28.416144,22.775659,0.000000,-1.372335
CCHIL1U5 CBIN Curncy,0.571897,0.492307,0.397249,-0.051807
HGA Comdty,-0.828219,-0.917694,-1.152213,-1.381366
CL1 Comdty,-0.080764,-0.183811,-0.248392,-0.376023
VIX Index,0.350689,0.335249,0.076202,0.537526


#Funcion Generadora de Rezagos

Esta función nos permitirá generar rezagos de las variables disponibles en la base de datos.


In [0]:
def crealags(base,lag_ini,nrolags):
    for lags in range(lag_ini,  nrolags+lag_ini): #Parte del rezago que definamos, no desde 1
        slag=base.shift(lags).copy(True)
        slag.columns=[str(col) + '_lag'+str(lags) for col in base.columns]
        if lags==lag_ini:#Bloque de datos inicial
            rezagos=pd.concat([slag], axis=1).copy(True)#genera primer bloque de datos 
        else: rezagos=pd.concat([rezagos,slag], axis=1).copy(True) #genera el resto del bloque de datos        
    return rezagos

##Generando Rezagos en la Data

Haciendo uso de la función recién generada, generaremos rezagos para cada una de las variables del dataframe que hemos importado desde excel. Para cada variable (incluye la dependiente), se han generado 5 rezagos.

In [63]:
nrolags=5
lag_ini = 1
rezagos = crealags(data,lag_ini,nrolags)
rezagos.head(10)

,USDCLP Curncy_lag1,USSW1 Curncy_lag1,USSW10 Curncy_lag1,CHSWP1 ICCH Curncy_lag1,CHSWP10 Curncy_lag1,CCHIL1U5 CBIN Curncy_lag1,HGA Comdty_lag1,CL1 Comdty_lag1,VIX Index_lag1,JPY Curncy_lag1,...,USDCLP Curncy_lag5,USSW1 Curncy_lag5,USSW10 Curncy_lag5,CHSWP1 ICCH Curncy_lag5,CHSWP10 Curncy_lag5,CCHIL1U5 CBIN Curncy_lag5,HGA Comdty_lag5,CL1 Comdty_lag5,VIX Index_lag5,JPY Curncy_lag5
2012-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-12-31,479.470,0.3340,1.7630,5.23,5.47000,72.603,344.35,90.80,22.72,85.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01,476.404,0.3240,1.8395,5.26,5.47000,72.479,350.65,91.82,18.02,86.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,475.382,0.3210,1.8395,5.27,5.47000,72.136,354.90,92.47,16.35,86.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,474.360,0.3205,1.8781,5.28,5.50000,70.128,359.15,93.12,14.68,87.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,472.420,0.3215,1.9455,5.28,5.53000,67.005,357.35,92.92,14.56,87.24,...,479.470,0.3340,1.7630,5.23,5.47,72.603,344.35,90.80,22.72,85.96
2013-01-07,473.120,0.3300,1.9360,5.27,5.55000,66.833,355.30,93.09,13.83,88.15,...,476.404,0.3240,1.8395,5.26,5.47,72.479,350.65,91.82,18.02,86.75
2013-01-08,471.340,0.3280,1.9270,5.28,5.58000,68.502,354.05,93.19,13.79,87.79,...,475.382,0.3210,1.8395,5.27,5.47,72.136,354.90,92.47,16.35,86.70
2013-01-09,472.250,0.3230,1.8910,5.27,5.55499,69.658,353.75,93.15,13.62,87.05,...,474.360,0.3205,1.8781,5.28,5.50,70.128,359.15,93.12,14.68,87.34
2013-01-10,470.950,0.3140,1.8875,5.25,5.54000,68.333,353.90,93.10,13.81,87.88,...,472.420,0.3215,1.9455,5.28,5.53,67.005,357.35,92.92,14.56,87.24


##Muestras training y testing con rezagos

In [0]:
#Separando Explicada de Explicativas
y= data_rezagos.iloc[:,0]
x= data_rezagos.iloc[:,1:-1]
#Umbral de Corte
year_corte = '2016-06-19 00:00:00'
#Trainig
x_tr =x[x.index<year_corte].copy(True)
y_tr =y[y.index<year_corte].copy(True)
#Testing
x_tst =x[x.index>year_corte].copy(True)
y_tst =y[y.index>year_corte].copy(True)

##Concatenando Rezagos con Data Original

Lo que ahora haremos será concatenar ambos DataFrame, eliminando en el proceso, todos los datos que perdimos al realizar los rezagos de las variables (**.dropna()**)

In [0]:
data_rezagos=pd.concat([data.iloc[:,0],rezagos],axis=1).dropna()

##Declaracion de Modelos a Estimar (forma conveniente)

En esta función, declararemos todos los modelos a utilizar, dentro de listas que ocuparemos más adelante.

In [66]:
def creamodelos(): 
    print('Creando Modelos...'.format())+'\n' 
    names = ["LinearRegression",
             'Ridge',
             'Lasso',
             'ElasticNet'
             ]

    regresores=[
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    ]
    
    print('Lista de modelos creados...'.format())+'\n' 
    print(names)
    print('\n' )
    return names, regresores

names, regresores  = creamodelos()

Creando Modelos...

Lista de modelos creados...

['LinearRegression', 'Ridge', 'Lasso', 'ElasticNet']




# Estimación + Métricas Training

Para ajustar los modelos que hemos declarado anteriormente (**creamodelos**), haremos uso de un loop que irá tomando cada uno de estos modelos declarados, estimándolo y simultáneamente calculando sus métricas de desempeño en un DataFrame que consolida toda la información.

In [67]:
modelos=[]
metricas_training=[]
counter=1
for name, modelo in zip(names, regresores):
    #print("entrenando modelo")
    #print(name)

    modelo.fit(x_tr,y_tr)
             
    #guarda el objeto modelo entrenado y colocale un id counter
    modelos.append({'algo':modelo,'idmod':counter})
    pred_training=modelo.predict(x_tr) 
    
    def metricas(y_real,pred,counter,name):
        res=[]
        #%%Creacion y exportacion de métricas del modelo.   
        MAE=mean_absolute_error(y_real,pred)
        RMSE=(mean_squared_error(y_real,pred))**0.5
        
        res.append((counter,name,MAE,RMSE))
        return res
    
    res_tr=metricas(y_tr,pred_training,counter,name)
    metricas_training.append(res_tr)
    counter=counter+1
    
    
from itertools import chain
def flatten(listOfLists):
    "Flatten one level of nesting"
    return chain.from_iterable(listOfLists)

metricas_training=list(flatten(metricas_training))
resultados_training=pd.DataFrame(data=metricas_training, columns=['id_modelo','Modelo','MAE','RMSE'])
resultados_training    

,id_modelo,Modelo,MAE,RMSE
0,1,LinearRegression,2.509078,3.298420
1,2,Ridge,2.522812,3.325773
2,3,Lasso,2.590847,3.460958
3,4,ElasticNet,2.570841,3.426490


#Estimacion + Métricas Testing

Repetimos el mismo proceso para la muestra de testing.

In [68]:
modelos=[]
metricas_testing=[]
counter=1
for name, modelo in zip(names, regresores):
    #print("entrenando modelo")
    #print(name)

    modelo.fit(x_tst,y_tst)
             
    #guarda el objeto modelo entrenado y colocale un id counter
    modelos.append({'algo':modelo,'idmod':counter})
    pred_testing=modelo.predict(x_tst) 
    
    def metricas(y_real,pred,counter,name):
        res=[]
        #%%Creacion y exportacion de métricas del modelo.   
        MAE=mean_absolute_error(y_real,pred)
        RMSE=(mean_squared_error(y_real,pred))**0.5
        
        res.append((counter,name,MAE,RMSE))
        return res
    
    res_tst=metricas(y_tst,pred_testing,counter,name)
    metricas_testing.append(res_tst)
    counter=counter+1
    


metricas_testing=list(flatten(metricas_testing))
resultados_testing=pd.DataFrame(data=metricas_testing, columns=['id_modelo','Modelo','MAE','RMSE'])
resultados_testing


,id_modelo,Modelo,MAE,RMSE
0,1,LinearRegression,2.504327,3.113960
1,2,Ridge,2.583467,3.207278
2,3,Lasso,2.700835,3.470332
3,4,ElasticNet,2.675898,3.435986
